In [ ]:
import scipy.ndimage
from numpy.linalg import norm
import pandas as pd
from joblib import Parallel, delayed
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
np.random.seed(100)

d = 256

s1 = pi # width of the gaussian

t = arange(- d / 2, d / 2)
h = (1 - t ** 2 / s1 ** 2) * exp(- (t ** 2) / (2 * s1 ** 2))
h = h - h.mean()

h_tf = fft.fft(fft.fftshift(h))
opA = lambda u : real(fft.ifft(fft.fft(u) * h_tf))

L = np.sqrt(np.sum(np.abs(h_tf)**2)) # Operator norm of opA
s = 16 # number of nonzero elements of xsharp

def training_set(n, var):
    x_train = zeros((d, n))
    eps_train = zeros((d, n))        
    y_train = zeros((d, n))
    for i in range(n):
        nonzero_indices = random.choice(d, s, replace=False)
        nonzero_values = random.uniform(-1, 1, s)
        x0 = zeros(d)
        x0[nonzero_indices] = nonzero_values
        x_train[:, i] = x0 / norm(x0, ord=2)
        
        eps_train[:, i] = random.normal(0, var, d)

        y_train[:, i] =  opA(x_train[:, i]) + eps_train[:, i]

    return x_train, y_train


In [ ]:
# We compute here D_||.||_1(x, y). Inputs vectors/matrices, outputs number
def breg_dist_l1(x, y): 
    norm_x = norm(x, ord=1)
    dot_prod = np.dot(np.sign(y), x)
    return norm_x - dot_prod

beta = abs(fft.fft(h)).max() ** 2
gamma = 1.3 / beta

# soft thresholding function here:
def st(x, Lambda):
    y = x - x / np.maximum(np.abs(x) / (Lambda * gamma), 1) 
    return y 

def fista(y, lam, warm_start_w=None):
    w = warm_start_w.copy() if warm_start_w is not None else np.zeros(len(y))
    z = zeros(len(y))
    nor1 = np.inf
    it = 0
    t_new = 1
    while nor1 >= 1e-6:
        t_old = t_new
        t_new = (1 + np.sqrt(1 + 4 * t_old ** 2)) / 2
        w_old = w.copy()
        z -= gamma * opA(opA(z) - y) / L
        w = st(z, lam / L)
        z = w + (t_old - 1.) / t_new * (w - w_old)
        nor1 = norm(w - w_old, ord=1)
        it += 1
    return w, it


In [ ]:
#Training functions
def train_l1(x_tr, y_tr, la):
    n_tr = x_tr.shape[1]
    distances = []
    for i in range(n_tr):
        f_lasso = fista(y_tr[:, i], la)[0]
        err = breg_dist_l1(x_tr[:, i], f_lasso)
        distances.append(err)
    return np.mean(distances)

def get_lambda(x_tr, y_tr, lamb): 
    num_cores = -1  #(-1 uses all available cores) 
    l1_err = Parallel(n_jobs=num_cores)(delayed(train_l1)(x_tr, y_tr, la) for la in lamb)
    l1_err = np.array(l1_err)
    min_index = np.argmin(l1_err)
    return lamb[min_index], l1_err.min()


In [ ]:
#debl1 plot

lamb = logspace(-2, 0, num=50)
std = 0.25
N_big = 1000
x_big, y_big = training_set(N_big, std)

N_vec = arange(5, 51, 5)
n_it = 30  # The perfect number of iterations is 30

L_at_lhatn = zeros((len(N_vec), n_it))
lamb_hatn = zeros((len(N_vec), n_it))

for i in range(len(N_vec)):
    print('n=', N_vec[i])          
    for j in range(n_it):
        xsmall, ysmall = training_set(N_vec[i], std)
        lamb_hatn[i, j] = get_lambda(xsmall, ysmall, lamb)[0]
        L_at_lhatn[i, j] = train_l1(x_big, y_big, lamb_hatn[i, j])


In [ ]:
meanlasso = mean(L_at_lhatn, axis=1)
lowerlasso = quantile(L_at_lhatn, 0.05, axis=1)
upperlasso = quantile(L_at_lhatn, 0.95, axis=1)

# Figures

plt.close('all')
fig, ax1 = plt.subplots(figsize=(20, 5), dpi=300)
ax1.plot(N_vec, meanlasso, '-')
ax1.scatter(N_vec, meanlasso, color='red', s=50)
ax1.fill_between(N_vec, lowerlasso, upperlasso, alpha=0.2)
ax1.set_ylabel(r'$L(X_{\widehat{\lambda}(n)})$', fontsize=25)
ax1.set_xlabel(r'$n$', fontsize=25)
ax1.set_xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.savefig("./debl1_tau=0.25.pdf", bbox_inches='tight')  # This shud go before show
plt.show(block=False)

In [ ]:
#debl2 plot
lamb1 = np.logspace(-2, 1, num=50)
lamb2 = np.logspace(-2, 1, num=10)
tau = np.logspace(-1, 0, num=30)

n_it = 30
bign = 500
smalln = 5

bign = 500
smalln = 5
n_it = 30


In [ ]:
lamb_star = np.zeros(len(tau))
Lstar = np.zeros(len(tau))

lamb_hat = np.zeros((len(tau), n_it))
Lhat = np.zeros((len(tau), n_it))

for i in range(len(tau)):
    print('tau=', tau[i])
    x_bign, y_bign = training_set(bign, tau[i])
    lamb_star[i], Lstar[i] = get_lambda(x_bign, y_bign, lamb1)
    for j in range(n_it):
        x_smalln, y_smalln = training_set(smalln, tau[i])
        lamb_hat[i, j] = get_lambda(x_smalln, y_smalln, lamb2)[0] 
        Lhat[i, j] = train_l1(x_bign, y_bign, lamb_hat[i, j])

In [ ]:
meandbl = np.mean(Lhat, axis=1) 
lowerdbl = np.quantile(Lhat, 0.05, axis=1) 
upperdbl = np.quantile(Lhat, 0.95, axis=1) 

plt.close('all')

fig, ax1 = plt.subplots(1, figsize=(20, 5), dpi=300)

ax1.plot(tau, meandbl, linestyle='-', label=r'$L(X_{\widehat{\lambda}_\Lambda(\tau)})$')
ax1.fill_between(tau, lowerdbl, upperdbl, alpha=0.2)
ax1.plot(tau, Lstar, linestyle='-', label=r'$L(X_{\widehat{\lambda}_\Lambda(\tau)})$')
ax1.legend(fontsize=20)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.tick_params(axis='both', which='major', labelsize=20)
ax1.tick_params(axis='both', which='minor', labelsize=20)

fig.supxlabel(r"$\tau$", fontsize=25)
plt.tight_layout()
plt.savefig("./debl2.pdf", bbox_inches='tight')
plt.show(block='False')

In [ ]:
# We test FISTA here with la^, selected with 100 points.

In [ ]:
x_train, y_train = training_set(100, var = 0.1)
lamb = logspace(-2, 0, num=50)
lambda_hat = get_lambda(x_train, y_train, lamb)[0]

In [ ]:
N_test = 5
x_test, y_test = training_set(N_test, var=0.1)

x = x_test[:, 0]
y = y_test[:, 0]
y_lasso = fista(y, lambda_hat)[0]

fig, (ax_orig, ax_filtered, ax_rec) = plt.subplots(3, 1, sharex=True, figsize=(20, 8), dpi=300)
ax_orig.stem(x)
ax_orig.set_title('Original', fontsize=25)
ax_orig.set_xlim(0, d - 1)
ax_orig.tick_params(axis='y', labelsize=20)
ax_filtered.stem(y)
ax_filtered.set_title('Blurred, noisy', fontsize=25)
ax_filtered.margins(0, 0.1)
ax_filtered.tick_params(axis='y', labelsize=20)
ax_rec.stem(y_lasso)
ax_rec.set_title('Recovered', fontsize=25)
ax_rec.margins(0, 0.1)
ax_rec.tick_params(axis='y', labelsize=20)
plt.subplots_adjust(hspace=0.5) 
plt.xticks(fontsize=20)
plt.savefig("./debl3.pdf", bbox_inches='tight')
plt.tight_layout()
plt.show()